In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# Read in training set and preprocess data

Delete columns wich contain string values.

Create training set where columns with missing values are dropped and where they are imputed.

In [2]:
###### Read the data
train = pd.read_csv('original/train.csv')
#train.head(n=3)
###### Exclude non numeric values
train = train.select_dtypes(exclude=['object'])
# drop 43 columns (81 -38)
###### Drop columns with missing values
cols_na = [col for col in train.columns 
                                 if train[col].isnull().any()]
train_nona = train.drop(cols_na, axis=1)
#train_nona = train.dropna(axis=1)
###### 
#train.shape, train_nona.shape
# 3 columns dropped, 38 -> 35

###### Make default imputations
my_imputer = Imputer()
train_impute = my_imputer.fit_transform(train)
train_impute = pd.DataFrame(train_impute)
train_impute.columns = train.columns
#train_nona.shape

Check for NaN values in test set and delete columns in both sets

In [3]:
# Read the test data
test = pd.read_csv('original/test.csv')
###### Exclude non numeric values
test = test.select_dtypes(exclude=['object'])
### Drop columns with non numeric entries and missing values
test_nona = test[train_nona.columns[0:34]]
#### Look for missing values
#print(test_nona.isnull().sum())
###### Drop columns with missing values
cols_na = [col for col in test_nona.columns 
                                 if test_nona[col].isnull().any()]
test_nona = test_nona.drop(cols_na, axis=1)
train_nona = train_nona.drop(cols_na, axis=1)
#test_nona.shape, train_nona.shape 

Impute values in test data

In [4]:
#### Impute values in test data
test_impute = my_imputer.fit_transform(test)
test_impute = pd.DataFrame(test_impute)
test_impute.columns = test.columns

# Find good parameters on training set 

In [5]:
########
y_nona = train_nona.SalePrice
X_nona = train_nona.drop(['SalePrice'], axis=1)
########
y_impute = train_impute.SalePrice
X_impute = train_impute.drop(['SalePrice'], axis=1)
########

what parameters to change

in randomforestregressor ?

in columns for prediction ?

In [6]:
## score dataset
def calc_mae(X, y, seed):
    train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = seed)
    # run ml
    model = RandomForestRegressor()
    model.fit(train_X, train_y)
    # pred values
    pred_val = model.predict(val_X)
    # calc accuracy
    mae = mean_absolute_error(val_y, pred_val)
    return mae

In [9]:
max = 100
mae1 = 0
mae2 = 0
for i in range (0, max):
    mae1 = mae1 + calc_mae(X_nona, y_nona, i)
    mae2 = mae2 + calc_mae(X_impute, y_impute, i)
print(max,mae1/max, mae2/max)

(100, 20968.579093150685, 19645.275191780816)


# Calculate Final model on whole training set

In [20]:
# pull data into target (y) and predictors (X)
train_y = train_impute.SalePrice
#predictor_cols = ['LotArea', 'OverallQual', 'YearBuilt', 'TotRmsAbvGrd']

# Create training predictors data
train_X = train_impute.drop(['SalePrice'], axis=1)
#train_X = train_impute[predictor_cols]

my_model = RandomForestRegressor()
my_model.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

# Read in test set, make predictions and write values to csv

In [22]:
# Treat the test data in the same way as training data. In this case, pull same columns.
#test_X = test[predictor_cols]
test_X = test_impute
# Use the model to make predictions
predicted_prices = my_model.predict(test_X)
# We will look at the predicted prices to ensure we have something sensible.
print(predicted_prices)

[ 129750.  154065.  168200. ...,  162900.  114090.  232529.]


In [23]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})
my_submission.to_csv('submission.csv', index=False)